# Figure 3.5 - Variable Order Approximation

### Design

In [1]:
import time as tm
import numpy as np

In [2]:
t0 = tm.time()
a = [i for i in np.arange(1,10+1)]
t1 = tm.time()
a, t1-t0

([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 0.00017189979553222656)

In [3]:
sample_n = 200
X = sorted(np.random.uniform(-2,2,sample_n))
X = np.reshape(X,(sample_n,1))
m = lambda x : x + 2*np.exp(-16*x**2)
Y = m(X) + np.reshape(np.random.normal(0,.5,sample_n),(sample_n,1))
X.shape, Y.shape

((200, 1), (200, 1))

### functions

In [6]:
Kh = lambda u,h : 3*(1-(u/h)**2)/4 * (1>=(u/h)**2)

mm2 = lambda A,B : np.matmul(A,B)
mm3 = lambda A,B,C : mm2(mm2(A,B),C)
mt = lambda A : np.transpose(A)
miv = lambda A : np.linalg.inv(A)

def mxgen(nr,nc,FUN) :
    r, c = np.arange(1,nr+1)[:,None], np.arange(1,nc+1)
    return(FUN(r,c))

In [7]:
def estm(h,p,estm_Y,estm_X,estm_x0) :
    FUN = lambda i,j : ((estm_X[i-1,:1] - estm_x0)**(j - 1)).squeeze()
    bf_X = mxgen(sample_n, p + 1, FUN)
    W = np.diag(Kh(estm_X - estm_x0,h).squeeze())
    Sn = mm3(mt(bf_X),W,bf_X)
    hatbeta = mm2(miv(Sn), mm3(mt(bf_X),W,estm_Y))
    return(hatbeta)
estm(.2,3,Y,X,0)
def estm_v(h,p,estm_Y,estm_X,estm_x0v) :
    return(np.asarray([estm(h,p,estm_Y,estm_X,xj).squeeze() for xj in estm_x0v]))

estm_v(.2,3,Y,X,np.linspace(-1.5,1.5,101)).shape

(101, 4)

$\left(\begin{array}{cccc} \hat{\beta}_0(x_1) & \hat{\beta}_1(x_1) & \cdots & \hat{\beta}_p(x_1) \\
\vdots & \vdots & \vdots & \vdots \\
\hat{\beta}_0(x_J) & \hat{\beta}_1(x_J) & \cdots & \hat{\beta}_p(x_J)
\end{array} \right)_{J\,\times\,(p+1)}$

In [9]:
# MSE ---------------------------------------------------------------------

def MSE(MSE_x0,MSE_h,MSE_nu,MSE_p,MSE_X,MSE_Y,a=2) :
#     MSE_x0 = 0 ; MSE_h = .3 ; MSE_nu = 0 ; MSE_p = 3 ; MSE_X = X ; MSE_Y = Y

#     functions
    MSE_Kh = lambda u : Kh(u,h = MSE_h)

    Snj = lambda j : sum(MSE_Kh(MSE_X - MSE_x0)*(MSE_X - MSE_x0)**j)
#     Snj = np.vectorize(Snj)
# #     ---------
    FUN = lambda i,j : ((MSE_X[i-1]-MSE_x0)**(j-1)).squeeze()
    bf_X = mxgen(sample_n, MSE_p+1, FUN)
    W = np.diag(MSE_Kh(MSE_X - MSE_x0).squeeze())
    Sn = mm3(mt(bf_X),W,bf_X)
    bf_X_a = mxgen(sample_n, MSE_p+a+1, FUN)
    beta_a = estm(MSE_h,MSE_p+a,MSE_Y,MSE_X,MSE_x0)
    tmp1 = mxgen(MSE_p+1,a,
                 lambda i,j : Snj(MSE_p+i+j-1)*(not(MSE_p+i+j-1 in np.linspace(MSE_p+a+1,MSE_p+a+MSE_p,MSE_p))))
#     tmp2 = beta_a[(MSE_p+2-1):(MSE_p+a+1)]
#     bias = mm(mm(miv(Sn),tmp1),tmp2)
#     bnu = bias[MSE_nu]
#     hY = estm_v(MSE_h,MSE_p,MSE_Y,MSE_X,MSE_X)[:,0:1]

#     sgsqn = np.sum((MSE_Y - hY)**2*MSE_Kh(MSE_X - MSE_x0))
#     sgsqdn = np.sum(np.diag(W - mm(
#         mm(mm(W,bf_X_a),miv(mm(mm(mt(bf_X_a),W),bf_X_a))),mm(mt(bf_X_a),W)
#     )))
    return()
#     Vnu = (chol2inv(chol(Sn)) %*% (t(bf_X) %*% W^2 %*% bf_X) %*% chol2inv(chol(Sn)))[MSE_nu+1,MSE_nu+1] * sgsq
#     return(bnu^2 + Vnu)
  
MSE(0, .3, 0, 3, X, Y)
# MSE = Vectorize(MSE,'MSE_x0')
# MSE = Vectorize(MSE,'MSE_p')

ValueError: operands could not be broadcast together with shapes (200,1) (4,2) 

In [ ]:
ra=MSE_Y
rb=estm_v(MSE_h,MSE_p,MSE_Y,MSE_X,MSE_X)[:,0:1]
rb

In [ ]:
not (MSE_p+2-1 in np.linspace(MSE_p+2+1,MSE_p+2+MSE_p,MSE_p))

In [ ]:
# MSE ---------------------------------------------------------------------

def MSE(MSE_x0,MSE_h,MSE_nu,MSE_p,
        MSE_X,MSE_Y,a=2) :
#     MSE_x0 = 0 ; MSE_h = .3 ; MSE_nu = 0 ; MSE_p = 3 ; MSE_X = X ; MSE_Y = Y

#     functions
    MSE_Kh = lambda u : Kh(u,h = MSE_h)

    Snj = lambda j : sum(MSE_Kh(MSE_X - MSE_x0)*(MSE_X - MSE_x0)^j)
    Snj = np.vectorize(Snj)
#     ---------
    bf_X = mxgen(sample_n, MSE_p+1, function(i, j, X_tmp = MSE_X, x0_tmp = MSE_x0){(X_tmp[i]-x0_tmp)^(j-1)})
    W = diag(c(MSE_Kh(MSE_X-MSE_x0)))
    Sn = t(bf_X) %*% W %*% bf_X
    bf_X_a = mxgen(sample_n, MSE_p+a+1, function(i,j,X_tmp=MSE_X,x0_tmp=MSE_x0){(X_tmp[i]-x0_tmp)^(j-1)})
    beta_a = estm(MSE_h,MSE_p+a,MSE_Y,MSE_X,MSE_x0)
    bias = chol2inv(chol(Sn)) %*% 
    mxgen(MSE_p+1,a,function(i,j){Snj(MSE_p+i+j-1) * !((MSE_p+i+j-1) %in% (MSE_p+a+1):(MSE_p+a+MSE_p))}) %*% 
    beta_a[(MSE_p+2):(MSE_p+a+1),1]
    bnu = bias[MSE_nu + 1]
    hY = estm_v(MSE_h,MSE_p,MSE_Y,MSE_X,MSE_X)[1,]

    sgsq = sum((MSE_Y - hY)^2*MSE_Kh(MSE_X - MSE_x0)) /
    sum(diag(W - W %*% bf_X_a %*% chol2inv(chol(t(bf_X_a) %*% W %*% bf_X_a)) %*% t(bf_X_a) %*% W))
    Vnu = (chol2inv(chol(Sn)) %*% (t(bf_X) %*% W^2 %*% bf_X) %*% chol2inv(chol(Sn)))[MSE_nu+1,MSE_nu+1] * sgsq
    return(bnu^2 + Vnu)
  
MSE = Vectorize(MSE,'MSE_x0')
MSE = Vectorize(MSE,'MSE_p')

In [ ]:
# order selection ---------------------------------------------------------

hv <- c(.2,.3,.4,.6)

nu0 <- 0
x.grd <- -1.8 + .036*(0:100)

order_selector <- function(xj,R,h,os.X,os.Y,nu=nu0){
  tmp <- MSE(xj,h,nu,(nu + 1):R,os.X,os.Y)
  return(apply(tmp, 1, which.min))
}

setEPS()
postscript('Fig-3-5(400).eps',height = 10)
layout(matrix(c(1,1,2:5), 3, 2, byrow = TRUE))

plot(X,Y, pch = 16, cex = .4)
plot(function(u)m(u),-2,2,add = T)
for(j in 1:4){
  orders <- order_selector(xj = x.grd,nu = nu0,h = hv[j],R = 3,X,Y)
  hY <- x.grd
  for(i in 1:length(x.grd)){
    hY[i] <- estm(hv[j],orders[i],Y,X,x.grd[i])[1]
  };lines(x.grd,hY,lty=j+1)
} ; legend('topleft',legend = c('true',paste('h=',hv,sep = '')),lty = 1:5)

In [ ]:
# MADEr --------------------------------------------------------------------

sim.size <- 400
MADE <- function(a,b){abs(a-b)/sim.size}

for (h0 in hv) {
  MADE_adp <- MADE_f_lcs <- MADE_f_lln <- MADE_f_lqd <- MADE_f_lcb <- rep(0,length(x.grd))
  simi <- 1
  layout(matrix(c(1,2), ncol = 2))

  while(simi < sim.size){
    tic()
    X <- runif(sample.n,-2,2)
    Y <- m(X) + rnorm(sample.n,0,.5)
    X <- as.matrix(X) ; Y <- as.matrix(Y)

    possibleError <- tryCatch({
      MADE_f_lcs <- MADE_f_lcs + MADE(estm.v(h = h0,p = 0,Y,X,x.grd),m(x.grd))
      MADE_f_lln <- MADE_f_lln + MADE(estm.v(h = h0,p = 1,Y,X,x.grd)[1,],m(x.grd))
      MADE_f_lqd <- MADE_f_lqd + MADE(estm.v(h = h0,p = 2,Y,X,x.grd)[1,],m(x.grd))
      MADE_f_lcb <- MADE_f_lcb + MADE(estm.v(h = h0,p = 3,Y,X,x.grd)[1,],m(x.grd))

      orders <- order_selector(xj = x.grd,h = h0,R = 3,X,Y)
      hY <- x.grd
      for(i in 1:length(x.grd)){hY[i] <- estm(h0,orders[i],Y,X,x.grd[i])[1]}
      MADE_adp <- MADE_adp + MADE(hY,m(x.grd))
      
      plot(x.grd,MADE_f_lcs/MADE_adp,type = 'l',lty = 1,
           main = paste('Ratio of MADE for h=.',h0,sep = ''),ylim = c(0,3))
      lines(x.grd,MADE_f_lln/MADE_adp,lty = 2)
      lines(x.grd,MADE_f_lqd/MADE_adp,lty = 3)
      lines(x.grd,MADE_f_lcb/MADE_adp,lty = 4)
      abline(h=1)
      
      simi <- simi + 1
      print(simi/sim.size*100);toc()
    }, error = function(e) {print(e)}
    )
    if(inherits(possibleError, "error")) {next}
  }
  l <- list(MADE_adp=MADE_adp,MADE_f_lcs=MADE_f_lcs,MADE_f_lln=MADE_f_lln,
            MADE_f_lqd=MADE_f_lqd,MADE_f_lcb=MADE_f_lcb)
  write.csv(l,paste('MADEr_h0',h0*10,'times',sim.size,'.csv',sep = ''))
}

for (hi in c(2,3,4,6)) {
  l <- read.csv(paste('MADEr_h0',hi,'times400.csv',sep = ''))
  plot(x.grd,l$MADE_f_lcs/l$MADE_adp,type = 'l',lty = 1,
       main = paste('Ratio of MADE for h=.',hi,sep = ''),ylim = c(0,3))
  lines(x.grd,l$MADE_f_lln/l$MADE_adp,lty = 2)
  lines(x.grd,l$MADE_f_lqd/l$MADE_adp,lty = 3)
  lines(x.grd,l$MADE_f_lcb/l$MADE_adp,lty = 4)
  abline(h=1)
}

dev.off()